In [1]:
import os

import torch
import torch.optim as optim

from tddl.models.resnet import PA_ResNet18
from tddl.models.resnet_lr import low_rank_resnet18

/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.6/site-packages/tensorly/backend/pytorch_backend.py:200: UserWarning: You are using an old version of PyTorch (1.7.1+cu101). We recommend upgrading to a newest one, e.g. >1.8.0.
  warnings.warn(f'You are using an old version of PyTorch ({torch.__version__}). '


In [2]:
cuda = "7"
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = cuda

cpu = "8"
os.environ["MKL_NUM_THREADS"] = cpu
os.environ["NUMEXPR_NUM_THREADS"] = cpu
os.environ["OMP_NUM_THREADS"] = cpu

In [ ]:
!pwd

In [3]:
save_path = "/home/jetzeschuurman/gitProjects/phd/tddl/notebooks/tmp"

In [12]:
pretrained = "/local/jetzeschuurman/f_mnist/logs/parn_18_d0.5_256_sgd_l0.1_g0.1_sTrue/1633280228/cnn_best"

# load pretrained model
pretrained_model = torch.load(pretrained)
pretrained_optimizer = optim.SGD(
    pretrained_model.parameters(), 
    lr=1e-2, momentum=0.9, weight_decay=1e-4,
)

# safe model
model_name = "pretrained_model.pth"
torch.save(pretrained_model, os.path.join(save_path, model_name))

# safe state dict
checkpoint_name = "pretrained_checkpoint"
torch.save(
    (pretrained_model.state_dict(), pretrained_optimizer.state_dict()),
    os.path.join(save_path, checkpoint_name),
)

# load state dict
pretrained_model_state, pretrained_optimizer_state = torch.load(
    os.path.join(save_path, "pretrained_checkpoint")
)
pretrained_model_from_state = PA_ResNet18(
    num_classes = 10,
    nc=1,
)
pretrained_model_from_state.load_state_dict(pretrained_model_state)

<All keys matched successfully>

In [4]:
factorized = "/local/jetzeschuurman/f_mnist/logs/parn-td-(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)-tucker-0.5-dTrue-iFalse_bn_256_sgd_l0.01_g0.1_sTrue/1633342529/fact_model_best"

# load factorized model
factorized_model = torch.load(factorized)
factorized_optimizer = optim.SGD(
    factorized_model.parameters(), 
    lr=1e-2, momentum=0.9, weight_decay=1e-4,
)

In [ ]:

# safe factorized model
model_name = "factorized_model.pth"
torch.save(factorized_model, os.path.join(save_path, model_name))

# safe factorized checkpoint
fact_checkpoint_name = "factorized_checkpoint"
torch.save(
    (factorized_model.state_dict(), factorized_optimizer.state_dict()),
    os.path.join(save_path, fact_checkpoint_name),
)

# load state dict
fact_model_state, fact_optimizer_state = torch.load(
    os.path.join(save_path, "factorized_checkpoint")
)
# prep model to load statedict in
layers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
rank = 0.5
factorization = 'tucker'
decompose_weights=False
fact_model_from_state = low_rank_resnet18(
    layers, 
    rank=rank, 
    decompose_weights=False,
    factorization='tucker',
    init=None,
    pretrained_model=None,
)
fact_model_from_state.load_state_dict(fact_model_state)

In [14]:
factorized_model.load_state_dict(fact_model_state)

<All keys matched successfully>

In [5]:
import numpy as np

In [7]:
np.isnan(np.nan)

True